<a href="https://colab.research.google.com/github/IgnacioAntonio82/PROCESAMIENTO-DE-HABLA-/blob/main/Desafio_grupo_noticias_Aguilar_Ignacio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 8754
print(newsgroups_train.data[idx])


/(hudson)
/If someone inflicts pain on themselves, whether they enjoy it or not, they
/are hurting themselves.  They may be permanently damaging their body.

That is true.  It is also none of your business.  

Some people may also reason that by reading the bible and being a Xtian
you are permanently damaging your brain.  By your logic, it would be OK
for them to come into your home, take away your bible, and send you off
to "re-education camps" to save your mind from ruin.  Are you ready for
that?  





/(hudson)
/And why is there nothing wrong with it?  Because you say so?  Who gave you
/the authority to say that, and set the standard for morality?

Why?

Because: 
I am a living, thinking person able to make choices for myself.
I do not "need" you to show me what you think is the way; I have observed
too many errors in your thinking already to trust you to make up the
rules for me.

Because:
I set the standard for my *own* morality, and I permit you to do 
the same for yourself.  I

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
cossim

array([0.11252759, 0.09561582, 0.17267024, ..., 0.09162675, 0.1121114 ,
       0.03334953])

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.49040531, 0.48118373, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 8754,  6552, 10613, ...,  6988,  6980,  9520])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
mostsim

array([ 6552, 10613,  3616,  8726,  3902])

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.religion.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.religion.misc
talk.religion.misc
talk.religion.misc
talk.politics.mideast
talk.religion.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío


**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**No puedes usar la misma solución ya presentada por alguien en el foro antes que Ud. Es decir, sus 5 documentos al azar deben ser diferentes a los ya presentados, o las palabras que elija para el ejercicio 3 deben ser diferentes a las ya presentadas.**



In [ ]:
import random #para seleccionar documentos al azar.
import numpy as np
from sklearn.datasets import fetch_20newsgroups #carga el dataset de 20 Newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer #convierte texto a vectores numéricos usando TF-IDF.
from sklearn.metrics.pairwise import cosine_similarity #mide qué tan parecidos son dos vectores

# 1. Cargar dataset (20 newsgroups completo)
newsgroups = fetch_20newsgroups(subset="all")
docs = newsgroups.data #lista de textos
labels = newsgroups.target #categoría de cada documento
label_names = newsgroups.target_names #nombres de las categorías (ej: "comp.graphics", "sci.space", etc.).

# 2. Vectorizar con TF-IDF
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
#Convierte cada documento en un vector de 5000 dimensiones
#stop_words="english" filtra palabras comunes como the, is, and.
X = vectorizer.fit_transform(docs)

# 3. Seleccionar 5 documentos al azar
random.seed(123)  # fijamos semilla por reproducibilidad
sample_ids = random.sample(range(len(docs)), 5) #Elige 5 IDs de documentos distintos al azar

# 4. Calcular similaridad coseno
similarities = cosine_similarity(X)

# 5. Para cada documento elegido, mostrar sus 5 más similares
for idx in sample_ids:
    print("="*90)
    print(f"📄 Documento elegido (ID {idx})")
    print(f"Categoría: {label_names[labels[idx]]}\n")
    print(docs[idx][:400].replace("\n"," ") + "...\n")  # fragmento del texto

    # Ranking de similaridad
    sim_scores = list(enumerate(similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top5 = [doc_id for doc_id, score in sim_scores[1:6]]
    #Lista todos los demás con su score de similitud
    #Ordena de mayor a menor.
    #Se queda con los 5 más parecidos (excluyendo el mismo documento)

    print(">>> Documentos más similares:")
    for j in top5:
        print(f"- ID {j} | Similitud: {similarities[idx,j]:.3f} "
              f"| Categoría: {label_names[labels[j]]}")
        print("  ", docs[j][:200].replace("\n"," ") + "...\n")

📄 Documento elegido (ID 1715)
Categoría: comp.os.ms-windows.misc

From: eric@tgm.CAM.ORG (Eric Trepanier) Subject: More Cool BMP files?? Reply-To: eric@tgm.CAM.ORG Organization: Bell Sygma, Revenue Systems Development Lines: 27   In article <1993Apr17.023017.17301@gmuvax2.gmu.edu> rwang@gmuvax2.gmu.edu writes:   >   > Hi, everybody:  >     I guess my subject has said it all.  It is getting boring  > looking at those same old bmp files that came with Windows.  So...

>>> Documentos más similares:
- ID 14497 | Similitud: 0.661 | Categoría: comp.os.ms-windows.misc
   From: rwang@gmuvax2.gmu.edu (John Wang) Subject: More Cool BMP files?? Distribution: usa Organization: George Mason University, Fairfax, Virginia, USA Lines: 13  Hi, everybody:     I guess my subject ...

- ID 1305 | Similitud: 0.577 | Categoría: comp.os.ms-windows.misc
   From: rfweber@bcstec.ca.boeing.com (Robert F. Weber) Subject: Re: More Cool BMP files?? Article-I.D.: bcstec.C5wL0r.6MB Distribution: usa Organization: Boe



**2**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups #carga un dataset de artículos de foros de noticias
from sklearn.feature_extraction.text import TfidfVectorizer #convierte documentos en una matriz de TF-IDF
from sklearn.metrics.pairwise import cosine_similarity #mide la similaridad entre vectores usando el coseno

# 1. Cargar dataset
newsgroups = fetch_20newsgroups(subset="all")
docs = newsgroups.data

# 2. Vectorizar con TF-IDF
vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
#transforma los documentos en una matriz documento × término.
#Filtra stopwords en inglés
#Se queda con las 5000 palabras más frecuentes/relevantes
X = vectorizer.fit_transform(docs)
#vectorizer es un objeto de TfidfVectorizer, que convierte texto en vectores numéricos usando TF-IDF.
terms = vectorizer.get_feature_names_out()
#Devuelve un array con todas las palabras (términos) que están en la matriz X.
#Esto permite saber qué palabra corresponde a cada columna de X.

# 3. Transponer matriz (término-documento)
X_t = X.T  # ahora cada fila es una palabra

# 4. Calcular similaridad entre términos
similarities_terms = cosine_similarity(X_t)

# 5. Elegir manualmente 5 palabras interpretables
selected_words = ["hockey", "windows", "space", "jesus", "game"]

for word in selected_words: # Recorre cada palabra de la lista selected_words (por ejemplo "hockey", "windows", etc.) para analizarla una por una
    idx = np.where(terms == word)[0][0]
    #terms es un array con todas las palabras del vocabulario.
    #np.where(terms == word) devuelve un array con la posición(es) donde aparece la palabra.
    #[0][0] toma el primer índice (la posición de la palabra en la matriz TF-IDF).
    sim_scores = list(enumerate(similarities_terms[idx]))
    #similarities_terms es la matriz de similitud entre todas las palabras (término × término).
    #similarities_terms[idx] devuelve un vector con la similitud de "hockey" con cada palabra del vocabulario.
    #enumerate(...) crea pares (índice, similitud) para poder saber qué palabra corresponde a cada valor.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #Ordena los pares (índice, similitud) de mayor a menor según la similitud.
    #Así, las palabras más cercanas a "hockey" quedan al principio.
    top5 = [terms[j] for j, score in sim_scores[1:6]]
    #top5 contiene las 5 palabras más semánticamente similares a la palabra actual.

    print("="*80)
    print(f"🔹 Palabra: '{word}'")
    print("Palabras más similares:", top5)

🔹 Palabra: 'hockey'
Palabras más similares: ['espn', 'sport', 'nhl', 'game', 'team']
🔹 Palabra: 'windows'
Palabras más similares: ['dos', 'ms', 'os', 'run', 'file']
🔹 Palabra: 'space'
Palabras más similares: ['shuttle', 'sender', 'digest', 'sci', 'isu']
🔹 Palabra: 'jesus'
Palabras más similares: ['christ', 'god', 'disciples', 'christian', 'christians']
🔹 Palabra: 'game'
Palabras más similares: ['games', 'espn', 'hockey', 'baseball', 'scored']



**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [ ]:
from sklearn.datasets import fetch_20newsgroups #carga un dataset de noticias clasificadas en 20 categorías
from sklearn.feature_extraction.text import TfidfVectorizer #transforma texto en vectores numéricos usando TF-IDF
from sklearn.naive_bayes import MultinomialNB, ComplementNB #modelos de Naive Bayes para clasificación de texto
from sklearn.metrics import f1_score #métrica para evaluar desempeño de clasificación
from sklearn.pipeline import Pipeline #permite encadenar transformaciones (TF-IDF) y modelos.
import numpy as np

# 1. Cargar dataset
train = fetch_20newsgroups(subset="train") #Le dice a fetch_20newsgroups que cargue solo el conjunto de entrenamiento.
test = fetch_20newsgroups(subset="test")  #Le dice a fetch_20newsgroups que cargue solo el conjunto de prueba.

X_train, y_train = train.data, train.target
X_test, y_test = test.data, test.target
# extrae los datos y las etiquetas del dataset que cargaste con fetch_20newsgroups y los guarda en variables separadas para que sean más fáciles de usar

# 2. Definir combinaciones de parámetros a probar
vectorizer_params = [
    {"ngram_range": (1,1), "max_features": 20000},
    {"ngram_range": (1,2), "max_features": 30000},
    {"ngram_range": (1,2), "max_features": None},  # sin límite
]

# 2. Definir la prueba del modelo
models = [
    ("MultinomialNB", MultinomialNB),
    ("ComplementNB", ComplementNB),
]

alphas = [0.1, 0.5, 1.0]
#alpha es un parámetro de suavizado de Laplace
#Sirve para evitar probabilidades cero cuando una palabra que aparece en el test no estaba en el entrenamiento.

best_score = 0
best_config = None


for vec_params in vectorizer_params: #Recorre todas las combinaciones posibles
    for model_name, Model in models:
        for alpha in alphas:

            # pipeline vectorizador + modelo
            clf = Pipeline([
                ("tfidf", TfidfVectorizer(stop_words="english", **vec_params)),
                ("nb", Model(alpha=alpha))
            ])
            #Se crea un pipeline de scikit-learn
            #Vectorizador TF-IDF convierte el texto en un vector numérico usando los parámetros actuales (vec_params) y eliminando stopwords en inglés
            #Modelo Naive Bayes: instancia MultinomialNB o ComplementNB con el alpha actual.

            clf.fit(X_train, y_train) #Ajusta el pipeline a los datos de entrenamiento.
            y_pred = clf.predict(X_test) #Predice las etiquetas del conjunto de prueba

            score = f1_score(y_test, y_pred, average="macro")
            #Calcula la métrica F1

            print(f"{model_name}, alpha={alpha}, vec={vec_params} => F1-macro={score:.4f}")
            #Muestra por pantalla qué combinación se está probando y su puntuación F1

            if score > best_score:
                best_score = score
                best_config = (model_name, alpha, vec_params)
                #Si la combinación actual supera la mejor puntuación hasta ahora, guarda:
                #El score más alto (best_score)
                #La configuración que lo produjo (best_config)

  #Este código hace un grid search manual: prueba todas las combinaciones posibles de vectorizador, modelo y alpha, mide su desempeño con F1 y guarda la mejor configuración

print("="*90)
print("🏆 Mejor configuración encontrada:")
print(best_config, "con F1-macro =", best_score)

MultinomialNB, alpha=0.1, vec={'ngram_range': (1, 1), 'max_features': 20000} => F1-macro=0.8281
MultinomialNB, alpha=0.5, vec={'ngram_range': (1, 1), 'max_features': 20000} => F1-macro=0.8157
MultinomialNB, alpha=1.0, vec={'ngram_range': (1, 1), 'max_features': 20000} => F1-macro=0.8036
ComplementNB, alpha=0.1, vec={'ngram_range': (1, 1), 'max_features': 20000} => F1-macro=0.8148
ComplementNB, alpha=0.5, vec={'ngram_range': (1, 1), 'max_features': 20000} => F1-macro=0.8173
ComplementNB, alpha=1.0, vec={'ngram_range': (1, 1), 'max_features': 20000} => F1-macro=0.8162
MultinomialNB, alpha=0.1, vec={'ngram_range': (1, 2), 'max_features': 30000} => F1-macro=0.8051
MultinomialNB, alpha=0.5, vec={'ngram_range': (1, 2), 'max_features': 30000} => F1-macro=0.7950
MultinomialNB, alpha=1.0, vec={'ngram_range': (1, 2), 'max_features': 30000} => F1-macro=0.7900
ComplementNB, alpha=0.1, vec={'ngram_range': (1, 2), 'max_features': 30000} => F1-macro=0.8095
ComplementNB, alpha=0.5, vec={'ngram_range':